## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ilulissat,GL,69.2167,-51.1000,21.22,68,75,12.66,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,75.81,75,6,16.69,clear sky
2,2,Bredasdorp,ZA,-34.5322,20.0403,72.28,52,40,12.66,scattered clouds
3,3,Runcu,RO,45.1167,23.1333,44.37,90,100,2.24,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,72.81,66,100,15.59,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,75.81,75,6,16.69,clear sky
2,2,Bredasdorp,ZA,-34.5322,20.0403,72.28,52,40,12.66,scattered clouds
4,4,Hermanus,ZA,-34.4187,19.2345,72.81,66,100,15.59,overcast clouds
5,5,Avarua,CK,-21.2078,-159.7750,77.05,83,20,11.50,few clouds
19,19,Cayenne,GF,4.9333,-52.3333,82.09,85,90,1.14,overcast clouds
22,22,Georgetown,MY,5.4112,100.3354,80.55,84,20,1.99,few clouds
23,23,Kahului,US,20.8947,-156.4700,74.46,89,20,4.61,light rain
24,24,Kapaa,US,22.0752,-159.3190,76.98,83,90,1.01,light rain
27,27,Port Elizabeth,ZA,-33.9180,25.5701,72.10,60,20,12.66,few clouds
31,31,Namibe,AO,-15.1961,12.1522,78.30,64,13,13.29,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                199
City                   199
Country                198
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,75.81,clear sky,-23.1203,-134.9692,
2,Bredasdorp,ZA,72.28,scattered clouds,-34.5322,20.0403,
4,Hermanus,ZA,72.81,overcast clouds,-34.4187,19.2345,
5,Avarua,CK,77.05,few clouds,-21.2078,-159.7750,
19,Cayenne,GF,82.09,overcast clouds,4.9333,-52.3333,
22,Georgetown,MY,80.55,few clouds,5.4112,100.3354,
23,Kahului,US,74.46,light rain,20.8947,-156.4700,
24,Kapaa,US,76.98,light rain,22.0752,-159.3190,
27,Port Elizabeth,ZA,72.10,few clouds,-33.9180,25.5701,
31,Namibe,AO,78.30,few clouds,-15.1961,12.1522,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")    
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))